In [35]:
import boto3
import os
import configparser

In [28]:
!ls data

29_2150_compressed_GlobalLandTemperaturesByCity.csv.zip
29_2150_compressed_GlobalLandTemperaturesByCountry.csv.zip
29_2150_compressed_GlobalLandTemperaturesByMajorCity.csv.zip
29_2150_compressed_GlobalLandTemperaturesByState.csv.zip
I94_SAS_Labels_Descriptions.SAS
airport-codes_csv.csv
capstone-user.csv
datasets_29_2150_GlobalTemperatures.csv
immigration_data_sample.csv
sas_data
sub-est2019_all.csv
us-cities-demographics.csv


In [20]:
config_aws = configparser.ConfigParser()
config_aws.read_file(open('../aws_credentials.cfg'))

KEY                    = config_aws.get('AWS','KEY')
SECRET                 = config_aws.get('AWS','SECRET')
ARN                    = config_aws.get('AWS', 'ARN')

In [42]:
config_s3 = configparser.ConfigParser()
config_s3.read_file(open('../aws_setup.cfg'))

S3_REGION              = config_s3.get('S3', 'REGION')
S3_BUCKET              = config_s3.get('S3', 'NAME')

In [21]:
s3_client = boto3.client(
    's3',
    aws_access_key_id=KEY,
    aws_secret_access_key= SECRET
)

In [27]:
response = s3_client.create_bucket(
    ACL='public-read-write',
    Bucket=S3_BUCKET,
    CreateBucketConfiguration={
        'LocationConstraint': S3_REGION
    },
#     GrantFullControl='string',
#     GrantRead='string',
#     GrantReadACP='string',
#     GrantWrite='string',
#     GrantWriteACP='string',
#     ObjectLockEnabledForBucket=True|False
)

In [30]:
print(response)

{'ResponseMetadata': {'RequestId': '240C11DBCA15E2B4', 'HostId': '2Pp1a76WSSQIC0rTqaijJa2UvWOQwUKlytXvyMrsTIyEPsLEPY9eEYlgvF8DDbi/xrmqnpLZWR0=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': '2Pp1a76WSSQIC0rTqaijJa2UvWOQwUKlytXvyMrsTIyEPsLEPY9eEYlgvF8DDbi/xrmqnpLZWR0=', 'x-amz-request-id': '240C11DBCA15E2B4', 'date': 'Sat, 12 Sep 2020 16:10:58 GMT', 'location': 'http://immigrate-demographics-s3-1629.s3.amazonaws.com/', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'Location': 'http://immigrate-demographics-s3-1629.s3.amazonaws.com/'}


In [38]:
for root,dirs,files in os.walk('data/sas_data'):
    print("\n===")
    print(root, dirs, files)


===
data/sas_data ['.ipynb_checkpoints'] ['part-00003-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet', '.part-00003-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet.crc', '.part-00012-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet.crc', 'part-00008-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet', '.part-00011-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet.crc', 'part-00004-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet', 'part-00013-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet', '.part-00000-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet.crc', 'part-00005-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet', '._SUCCESS.crc', '.part-00008-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet.crc', 'part-00012-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet', '.part-00004-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snappy.parquet.crc', 'part-00002-b9542815-7a8d-45fc-9c67-c9c5007ad0d4-c000.snap

In [40]:
for root,dirs,files in os.walk('data/sas_data'):
    for file in files:
        upload_from = os.path.join(root, file)
        upload_to = os.path.join(root, file)
        s3_client.upload_file(upload_from, S3_NAME, upload_to)

In [43]:
## create log folder for EMR
# s3_path = f"s3://{S3_BUCKET}/logs"

In [47]:
s3_client.put_object(ACL='public-read-write',
                     Bucket=S3_BUCKET, 
                     Key='logs/')

{'ResponseMetadata': {'RequestId': '535F9514F9AB81BC',
  'HostId': 'qXaJOO6qRcjD1h+gwXooiEoxNWEpY9rU7hHgmSV02xsA/8A+oX680+3DSw/xd3xA90k1j+a+LEE=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'qXaJOO6qRcjD1h+gwXooiEoxNWEpY9rU7hHgmSV02xsA/8A+oX680+3DSw/xd3xA90k1j+a+LEE=',
   'x-amz-request-id': '535F9514F9AB81BC',
   'date': 'Sat, 12 Sep 2020 17:28:14 GMT',
   'etag': '"d41d8cd98f00b204e9800998ecf8427e"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"'}